In [1]:
import numpy as np
import random
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import spatial as sp

In [2]:
# Node and Linked List Classes

class Node:
    def __init__(self,initdata):
        self.data = initdata
        self.next = None

    def getData(self):
        return self.data

    def getNext(self):
        return self.next

    def setData(self,newdata):
        self.data = newdata

    def setNext(self,newnext):
        self.next = newnext


class OrderedList:
    def __init__(self):
        self.head = None

    def search(self,item):
        current = self.head
        found = False
        stop = False
        while current != None and not found and not stop:
            if current.getData() == item:
                found = True
            else:
                if current.getData() > item:
                    stop = True
                else:
                    current = current.getNext()

        return found
    
    def show(self):
        current = self.head
        while current != None:
            print current.getData()
            current = current.getNext()        
    
    def add(self,item):
        current = self.head
        previous = None
        stop = False
        while current != None and not stop:
            if current.getData() > item:
                stop = True
            else:
                previous = current
                current = current.getNext()

        temp = Node(item)
        if previous == None:
            temp.setNext(self.head)
            self.head = temp
        else:
            temp.setNext(current)
            previous.setNext(temp)
            
    def remove_first(self):
        self.head = self.head.getNext()
            

    def isEmpty(self):
        return self.head == None

    def size(self):
        current = self.head
        count = 0
        while current != None:
            count = count + 1
            current = current.getNext()

        return count

### Fixed variables:

In [3]:
GHZ_inner     = 20000.  # radio interno de la zona galactica habitable, años luz
GHZ_outer     = 60000.  # radio interno de la zona galactica habitable, años luz
tau_awakening = 10000.  # tiempo medio, en años, que hay que esperar para que aparezca otra CETI en la galaxia
tau_survive   = 5000.   # Tiempo medio, en años, durante el cual una CETI esta activa
D_max         = 3000.   # Maxima distancia, en años luz, a la cual una CETI puede enviar o recibir mensajes

### Simulation variables

In [4]:
# Inicializar listas de datos

# lista de CETIs alguna vez activas: dictionary
CETIs = dict()

# lista de CETIs actualmente activas: ndarray
CHATs = []

# inicializacion del tiempo: scalar
t_now = 0

# inicializacion del ID: index
ID = 0

# lista de tiempos de eventos futuros: ordered list
t_forthcoming = OrderedList()

# maximo tiempo para simular
t_max = 1000.

In [6]:
t = 0.

while (t<t_max):
    
    try:
        t_forthcoming
    except NameError:
        next_event = t_forthcoming.head.getData()
    else:
        next_event = [0, 0, None, 3]

        
case = next_event[3]    
    
if case==1:

    # actualizar el ID
    ID = ID + 1

    # sortear el lugar donde aparece dentro de la GHZ
    r = np.sqrt(random.random()*GHZ_outer**2 + GHZ_inner**2)
    o = random.random()*2.*np.pi
    x = r * np.cos(o)  # X position on the galactic plane
    y = r * np.sin(o)  # Y position on the galactic plane

    # sortear el tiempo de actividad
    t_active = np.random.exponential(tau_survive, 1)[0]
    t_hola = t_now
    t_chau = t_hola + t_active

    # agregar el tiempo de desaparición a la lista de tiempos
    next_event = [t_chau, ID, None, 2]
    t_forthcoming.add(next_event)

    # agregar la CETI a la lista histórica
    #CETIs[ID].append( [(x, y, t_hola, t_chau)] )
    CETIs[ID] = (x, y, t_hola, t_chau)

    # sortear el tiempo de aparición de la próxima CETI
    t_next_awakening = np.random.exponential(tau_awakening, 1)
    next_event = [t_next_awakening, ID+1, None, 1]
    t_forthcoming.add(next_event)

    try:
        tree
    except NameError:
        print 'No existe'
    else:
        # calcular todas las CETIs dentro del radio
        query_point = [x,y]
        idx = tree.query_ball_point(query_point, r=D_max)
        
        query_point = [x,y]

        for k in idx:
            z = np.sqrt(((np.array(query_point) - np.array(CHATs[k]))**2).sum())
        
            # Agregar a t_forthcoming la lista de tiempos de contacto
        
            # 1.- Desde la recién agregada hacia las otras CETIs
            z = t_now + z
            next_event = [z, ID, k, 3]
            t_forthcoming.add(next_event)

            # 2.- Desde las otras CETIs hacia la reciés agregada
            t_hola = CETIs[2][2]
            t = z - t_hola
            next_event = [t, ID, k, 3]
            t_forthcoming.add(next_event)
 

    # agregar la CETI a la lista de CETIs activas
    # [ID, x, y, t_hola, t_chau]
    CHATs.append([x, y])
    
    # # rehacer el árbol
    tree = sp.cKDTree( data=CHATs ) 

    # eliminar el tiempo actual
    t_forthcoming.remove_first()
        
        
        
if case==2:


    # actualizar el ID
    # t_now y ID son conocidos

    # eliminar la CETI a la lista de CETIs activas
    # [ID, x, y, t_hola, t_chau]
    CHATs.pop(ID)

    # rehacer el árbol
    tree = sp.cKDTree( data=CHATs ) 

    # eliminar el tiempo actual
    t_forthcoming.remove_first()
    
    
if case==3:
    
    # eliminar la CETI a la lista de CETIs activas
    # [ID, x, y, t_hola, t_chau]
    ID = 1
    t_chau = t_now

    CETIs[ID] = [(x, y, t_hola, t_chau)]

    # rehacer el árbol
    M = np.column_stack([x, y])
    tree = sp.cKDTree( data=M ) 


    # eliminar el tiempo actual
    t_forthcoming.remove_first()
    
    
if case==4:
    

KeyboardInterrupt: 